In [ ]:
# !pip install datasets tokenizers transformers

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from transformers import get_linear_schedule_with_warmup
import datasets
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from collections import defaultdict
from torch.utils.data import Dataset,  DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device :', device)

In [ ]:
# Define all parameters
args = {'max_sentence': 16,
        'max_word': 32,
        'batch_size': 4,
        'device': device,
        'num_class': 5,
        'class_weights': torch.tensor([0.25, 0.25, 0.25, 0.25, 0.]),
        'unfreezed_bert_layer': 3, 'bert_output_size': 768,
        'lstm_hidden_dimension':768, 'lstm_layers':2,
        'dropout': 0.5, 'lr': 1e-4, 'epochs': 3,
        'dataset_percentage': '1%'
        }

# max_sentence_per_dialog = args['max_sentence']
# max_word_per_sentence = args['max_word']

In [ ]:
dataset_train , dataset_test = load_dataset("silicone","dyda_da",split=[f"train[:{args['dataset_percentage']}]",
                                                                        f"test[:{args['dataset_percentage']}]"])
dataset = datasets.DatasetDict({"train" : dataset_train,"test":dataset_test})
dataset = dataset.rename_column("Label", "labels")
dataset = dataset.rename_column("Utterance", "text")
idx2label = {idx:label for idx, label in enumerate(dataset['train'].features['labels'].names)}
idx2label

### Format data

In [ ]:
sns.histplot(np.unique(dataset['train'][:]['Dialogue_ID'], return_counts=True)[1])
plt.title('Histogram of number of sentences (uterances) per dialog')
plt.show()

In [ ]:
# define BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
vocab_stoi = tokenizer.get_vocab()

In [104]:
dataset['test']

Dataset({
    features: ['text', 'Dialogue_Act', 'Dialogue_ID', 'labels', 'Idx'],
    num_rows: 77
})

In [ ]:
def tokenize_pad_numericalize_dialog(entry, vocab_stoi, max_length=32):
  ''' message level '''
  dialog = [[vocab_stoi['[CLS]']]
            + [vocab_stoi[token] if token in vocab_stoi else vocab_stoi['[UNK]'] for token in tokenizer.tokenize(e.lower()) ] 
            + [vocab_stoi['[SEP]']]
            for e in entry]
  padded_dialog = list()
  for d in dialog:
    if len(d) < max_length: 
        padded_dialog.append(d + [ vocab_stoi['[PAD]'] for i in range(len(d), max_length)])
    elif len(d) > max_length:
        padded_dialog.append(d[:max_length])
    else:           
         padded_dialog.append(d) 
  
  return padded_dialog

def tokenize_all_dialog(entries, vocab_stoi, max_message_length=32,
                         max_dialog_length=16):
    ''' dialog level '''
    pad_message = [ vocab_stoi['[PAD]'] ]
    pad_label = [4] # because our labels go from 0 to 3
    res_dialog, res_labels = [], []

    # Group messages by dialogue ID
    dialogues = defaultdict(list)
    dialogue_labels = defaultdict(list)
    for i in range(len(entries['text'])):
        dialogue_id = entries['Dialogue_ID'][i]
        dialogues[dialogue_id].append(entries['text'][i])
        dialogue_labels[dialogue_id].append(entries['labels'][i])

    # Tokenize and pad messages for each dialogue
    for dialogue_id, text in dialogues.items():
        # entry = text.strip().split(' | ')[1:]  # Remove speaker ID
        labels = dialogue_labels[dialogue_id]
        text = tokenize_pad_numericalize_dialog(text, vocab_stoi, max_length=max_message_length)
        if len(text) < max_dialog_length:
            text = text + [[vocab_stoi['[PAD]']] * max_message_length for i in range(len(text), max_dialog_length)]
            labels = labels +pad_label * (max_dialog_length-len(labels))
        elif len(text) > max_dialog_length:
            text = text[-max_dialog_length:]
            labels = labels[-max_dialog_length:]
        res_dialog.append(text)
        res_labels.append(labels)

    res = {'text': res_dialog, 'labels': res_labels}
    return res

encoded_dataset = dataset.map(lambda e: tokenize_all_dialog(e, vocab_stoi), batched=True, 
                              remove_columns=['Dialogue_Act','Dialogue_ID','Idx'])
encoded_dataset.set_format("torch")

In [109]:
class DialogActDataset(Dataset):
    def __init__(self, data, args, shuffle_test=False):
      self.args = args
      self.data = data
      self.shuffle_test = shuffle_test

    # Shuffle sentences within each dialogue in the test dataset
      if self.shuffle_test:
          for i in range(len(self.data)):
              idx = np.random.permutation(self.data[i]['text'].shape[0])
              self.data[i]['text'] = self.data[i]['text'][idx, :]
              self.data[i]['labels'] = self.data[i]['labels'][idx]

    def __len__(self):
      return len(self.data)
    
    def __getitem__(self, idx):
      item = {
          "text": np.array(self.data[idx]['text']),
          "label": np.array(self.data[idx]['labels'])
      }
      return item

In [ ]:
# Convert the test set to a mutable dataset
test_dataset = encoded_dataset['test'][:]

# Shuffle sentences within each dialogue
for i in range(len(test_dataset)):
    idx = torch.randperm(test_dataset['text'].shape[1])
    test_dataset['text'][i] = test_dataset['text'][i][idx, :]

# Print the first sentence of the first dialogue before and after shuffling
print(encoded_dataset['test'][:]['text'][0][0])
print(test_dataset['text'][0][0])

In [112]:
train_loader = DataLoader(DialogActDataset(encoded_dataset['train'], args), batch_size=args['batch_size'], shuffle=True, drop_last=True)
test_loader  = DataLoader(DialogActDataset(encoded_dataset['test'], args, shuffle_test=False),
                                           batch_size=args['batch_size'], shuffle=True, drop_last=True)
shuffled_test_loader  = DataLoader(DialogActDataset(encoded_dataset['test'], args, shuffle_test=True),
                                   batch_size=args['batch_size'], shuffle=True, drop_last=True)


In [113]:
for element in test_loader:
    print(element)
    break

{'text': tensor([[[  101, 23998,  1299,  ...,     0,     0,     0],
         [  101,  1199,  1184,  ...,     0,     0,     0],
         [  101, 24195,   106,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]],

        [[  101,  1363,  2106,  ...,     0,     0,     0],
         [  101,  1363,  2106,  ...,     0,     0,     0],
         [  101,  1155,  1268,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]],

        [[  101,  1293,  1263,  ...,     0,     0,     0],
         [  101,   178,  1341,  ...,   119,   102,     0],
         [  101,  1115,   787,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
 

In [107]:
encoded_dataset['test']

Dataset({
    features: ['text', 'labels'],
    num_rows: 8
})

In [106]:
test_dataset

{'text': tensor([[[  101, 23998,  1299,  ...,     0,     0,     0],
          [    0,     0,     0,  ...,     0,     0,     0],
          [  101,  1202,  1128,  ...,     0,     0,     0],
          ...,
          [  101,  1185,   117,  ...,     0,     0,     0],
          [  101,   178,  1400,  ...,     0,     0,     0],
          [  101,  8147,   136,  ...,     0,     0,     0]],
 
         [[    0,     0,     0,  ...,     0,     0,     0],
          [    0,     0,     0,  ...,     0,     0,     0],
          [    0,     0,     0,  ...,     0,     0,     0],
          ...,
          [    0,     0,     0,  ...,     0,     0,     0],
          [    0,     0,     0,  ...,     0,     0,     0],
          [  101,  1184,  1111,  ...,     0,     0,     0]],
 
         [[  101,  1195,   112,  ...,     0,     0,     0],
          [  101,  1115,   112,  ...,     0,     0,     0],
          [  101,  2871,  1272,  ...,     0,     0,     0],
          ...,
          [    0,     0,     0,  ...,    

## define Model

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, args):
        super(BiLSTM, self).__init__()

        self.args = args # TODO : sortir du args ce qui est nécessaire
        self.bert_model = BertModel.from_pretrained('bert-base-cased')

        # Freeze some Bert Layers
        for param in self.bert_model.parameters():
            param.requires_grad = False
        # Unfreeze the last layer
        for param in self.bert_model.encoder.layer[-args['unfreezed_bert_layer']:].parameters():
            param.requires_grad = True

        # Dimensions are divided by 2 due to bidirectional being True
        self.lstm = nn.LSTM(input_size=args['bert_output_size'],
                            hidden_size=args['lstm_hidden_dimension']//2, 
                            num_layers=args['lstm_layers'],
                            batch_first=True,
                            bidirectional=True)
        
        self.drop = nn.Dropout(p=args['dropout'])
        self.hidden2intent = nn.Linear(args['lstm_hidden_dimension'], args['num_class'])

    def forward(self, batch, training=True):

        input_ids = batch['text'].to(args['device'])
        if training:
            labels = batch['label'].to(args['device'])

        input_ids = input_ids.view(args['batch_size']*args['max_sentence'], args['max_word'])
        outputs = self.bert_model(input_ids)['pooler_output']
        outputs = outputs.view(args['batch_size'], args['max_sentence'], -1)
 
        outputs = self.drop(outputs)
        outputs, _ = self.lstm(outputs)
        outputs = self.drop(outputs)
        outputs = self.hidden2intent(outputs)
        outputs = outputs.view(-1, args['num_class'])
        logits = F.log_softmax(outputs, 1)
        
        # reshape the predicted tags to follow the batch and sequence length
        # tag_seq = tag_seq.view(args['bsize'], args['max_sentence'])

        loss_fn = nn.NLLLoss(weight=self.args['class_weights'].to(args['device']))  
        loss = loss_fn(logits, labels.view(-1)) 

        # Compute accuracy without 4 labels (PAD)
        _, tag_seq  = torch.max(logits, 1)
        labels = labels.view(-1)
        tag_seq = tag_seq.view(-1)

        mask = labels != 4
        labels, tag_seq = labels[mask], tag_seq[mask]
        accuracy = (labels == tag_seq).sum().item() / len(labels)

        return loss, logits, tag_seq, accuracy

In [ ]:
model = BiLSTM(args).to(args['device'])

In [ ]:
def train(model, optimizer, epoch, train_loader):

  model.train()
  training_dict = {'loss': [], 'acc': []}

  for batch in tqdm(train_loader, desc="Training, Epoch %s:" % (epoch),
                    position=0 ,leave=True):
    optimizer.zero_grad()
    model.zero_grad()
    loss, logits, tag_seq, accuracy = model(batch)
    loss.backward()
    optimizer.step()
    training_dict['loss'].append(loss.item())
    training_dict['acc'].append(accuracy)
  
  avg_loss = sum(training_dict['loss'])/len(training_dict['loss'])
  avg_acc = sum(training_dict['acc'])/len(training_dict['acc'])
  return avg_loss, avg_acc

@torch.no_grad()
def test(model, epoch, test_loader):
  model.eval()
  test_dict = {'loss': [], 'acc': []}

  for batch in tqdm(test_loader, desc="Test, Epoch %s:" % (epoch), 
                    position=0 ,leave=True):
    loss, logits, tag_seq, accuracy = model(batch)
    test_dict['loss'].append(loss.item())
    test_dict['acc'].append(accuracy)
  
  avg_loss = sum(test_dict['loss'])/len(test_dict['loss'])
  avg_acc = sum(test_dict['acc'])/len(test_dict['acc'])

  return avg_loss, avg_acc

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
# Train the model
for epoch in range(args['epochs']):
    train_loss, train_acc = train(model, optimizer, epoch, train_loader)
    test_loss, test_acc = test(model, epoch, test_loader)
    print(f'Epoch {epoch + 1}/{args["epochs"]} - Train loss: {train_loss:.3f} - Test loss: {test_loss:.3f} - Train acc: {train_acc:.3f} - Test acc: {test_acc:.3f}')